In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.linear_model import LinearRegression
import math
import matplotlib.pyplot
from matplotlib import pyplot as plt
from sklearn.linear_model import BayesianRidge

In [4]:
trainingPercent = 90
testingPercent = 10
epochs = 10000
learningRate = 0.01

In [5]:
def readData(fileName):
    df = pd.read_csv(fileName)
    return df

In [6]:
df = readData("insurance.csv")

In [7]:
print(df.head())

   age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520


In [8]:
def encodeData(df):
    sex = df.iloc[:,:].values
    labelEncoder = LabelEncoder()
    sex[:,1] = labelEncoder.fit_transform(sex[:,1])
    step1 = pd.DataFrame(sex)
    
    smoker = step1.iloc[:,:].values
    smoker[:,4] = labelEncoder.fit_transform(smoker[:,4])
    step2 = pd.DataFrame(smoker)
    
    region = step2.iloc[:,:].values
    region[:,5] = labelEncoder.fit_transform(region[:,5])
    return pd.DataFrame(region)

In [9]:
df = encodeData(df)

In [10]:
def normalizeData(df):
    df_norm = (df - df.mean()) / (df.max() - df.min())
    return df_norm,df.mean(),df.max(),df.min()

In [24]:
df,mean,maximum,minimum = normalizeData(df)
print(mean)

0    5.974294e-18
1    3.435219e-17
2   -1.296505e-19
3    9.807800e-17
4   -4.211048e-18
5   -6.721081e-18
6   -2.800450e-18
dtype: float64


In [25]:
print(maximum)

0    0.538978
1    0.494768
2    0.604428
3    0.781016
4    0.795217
5    0.494768
6    0.806084
dtype: float64


In [26]:
print(minimum)

0   -0.461022
1   -0.505232
2   -0.395572
3   -0.218984
4   -0.204783
5   -0.505232
6   -0.193916
dtype: float64


In [12]:
def shuffleData(df):
    shuffled = df.sample(frac=1).reset_index(drop=True)
    return shuffled

In [13]:
df = shuffleData(df)

In [14]:
def getTrainingAndTesting(df,trainingPercent):
    percentData = int(math.ceil((len(df))*(trainingPercent*0.01)))
    trainingFeatures = df.iloc[0:percentData,0:5].values
    trainingTargets = df.iloc[0:percentData,6:7].values
    testingFeatures = df.iloc[percentData:df.shape[0],0:5].values
    testingTargets = df.iloc[percentData:df.shape[0],6:7].values
    return trainingFeatures,trainingTargets,testingFeatures,testingTargets

In [15]:
trF,trT,teF,teT = getTrainingAndTesting(df,trainingPercent)
print(trF.shape)

(1205, 5)


In [16]:
def getMatrices(x):
    ones = np.ones([x.shape[0],1])
    x = np.concatenate((ones,x),axis=1)
    weights = np.zeros([1,x.shape[1]])
    return x,weights

In [17]:
x,weights = getMatrices(trF)
teF,w = getMatrices(teF)
print(x[0])

[1.0 0.10419509975953722 0.4947683109118086 -0.2294430148234204
 0.18101644245142 -0.20478325859491778]


In [18]:
def costFunction(x,train_y,weights):
    error = np.power(((x @ weights.T)-train_y),2)
    temp = np.sum(error)/(2 * len(x))
    return math.sqrt(temp/len(train_y))

In [19]:
def gradientDescent(x,y,epochs,learningRate,weights):
    error = np.zeros(epochs)
    for i in range(epochs):
        weights = weights - (learningRate/len(x)) * np.sum(x * (x @ weights.T - y), axis=0)
        error[i] = costFunction(x, y, weights)
    return weights,error

In [20]:
finalWeights,error = gradientDescent(x,trT,epochs,learningRate,weights)


TypeError: Object arrays are not currently supported

In [ ]:
w = reg.coef_
print(w)

In [ ]:
b = reg.intercept_
print(b)

In [ ]:
finalWeights = [0.0008109,0.18873942,-0.00285686,0.19994857,0.03614326,0.38295293]